# Bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from datetime import date, datetime

# Sanity check



## Combinando as tabelas, filtrando o ESTADO e compactando

In [ ]:
nome_colunas = [
        "cnpj_basico",
        "cnpj_ordem",
        "cnpj_dv",
        "cod_identificador_matriz_filial",
        "nome_fantasia",
        "cod_situacao_cadastral",
        "data_situacao_cadastral",
        "cod_motivo_situacao_cadastral",
        "end_nome_cidade_no_exterior",
        "end_cod_pais",
        "data_inicio_atividade",
        "cod_cnae_fiscal_primaria",
        "cod_cnae_fiscal_secundaria",
        "end_tipo_de_logradouro",
        "end_logradouro",
        "end_numero",
        "end_complemento",
        "end_bairro",
        "end_cep",
        "end_uf",
        "end_cod_municipio",
        "ddd1",
        "telefone1",
        "ddd2",
        "telefone2",
        "dddfax",
        "fax",
        "email",
        "cod_situacao_especial",
        "data_situacao_especial"]

In [ ]:
#-----------------------------------------------------------> STEP: Juntar e compactar bases.
# Tempo p execução da celula: 11m
# 0) Lista com nome de todos arquivos contidos na pasta 1


pasta_arquivos_cnpj = "/content/drive/MyDrive/Base - CNPJ - 1 - Alterações"
arquivos = os.listdir(pasta_arquivos_cnpj)

cont = 0
endereço_arquivos = []

for i in arquivos:
    a = pasta_arquivos_cnpj + r"/" + i
    endereço_arquivos.append(a)

# 1) Pega tabelas originais, filtra o estado e salva na pasta 2 


for arquivo in endereço_arquivos:
    cnpj = pd.read_csv(arquivo, sep=";", names=nome_colunas, encoding="latin1", dtype="object",  compression="zip")
    #----------------------->  ESCOLHER ESTADO
    tabela_cnpj_filtrada = cnpj.loc[cnpj['end_uf'] == 'MT'] 
    cont = cont + 1
    tabela_cnpj_filtrada.to_csv("/content/drive/MyDrive/Base - CNPJ - 2 - (Filtrada)" + r"/" + "DF_filtrado" + str(cont) + ".csv", index=False)


# 2) Lista com nome de todos arquivos contidos na pasta 2, faz a junção de todas as tabelas e salva na pasta 3


juntar_bases_filtradas = "/content/drive/MyDrive/Base - CNPJ - 2 - (Filtrada)"
arquivos =  os.listdir(juntar_bases_filtradas)

endereço_bases_filtradas = []
for i in arquivos:
    a = juntar_bases_filtradas + r"/" + i
    endereço_bases_filtradas.append(a)

arquivo_principal = endereço_bases_filtradas[0]
base_principal = pd.read_csv(arquivo_principal, dtype="object")

for base in endereço_bases_filtradas[1:]:
    base_append = pd.read_csv(base, dtype="object")
    base_principal = pd.concat([base_principal,base_append],ignore_index=True)

base_principal.to_csv("/content/drive/MyDrive/Base - CNPJ - 3 - Final/Tabela_final.csv", index=False )

#----------------------------------------------------------------------------------------> REINICIAR KERNEL! MUITO USO DE MEMORIA RAM.

##Filtros gerais, Drop colunas e correção de tipos


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Base - CNPJ - 3 - Final/Tabela_final.csv", dtype="object")

In [ ]:
    #-------------> Cod 08 = Empresa BAIXADA, Cod 03 = Suspensa
    
    #df = df.loc[(df['cod_situacao_cadastral'] == '08') | (df['cod_situacao_cadastral'] == '03')]
    df = df.loc[(df['cod_situacao_cadastral'] == '08')]


    #------------> Colunas não necessárias ( Motivo: As colunas retiradas a baixo não ofereceram nenhuma informação adicional relevante para os fins das analises)

    df.drop(columns=['data_inicio_atividade','end_bairro','end_complemento','end_numero','end_logradouro','end_tipo_de_logradouro','cod_cnae_fiscal_secundaria','end_cod_pais','end_nome_cidade_no_exterior','ddd1','telefone1','ddd2','telefone2','dddfax','fax','email','cod_situacao_especial','data_situacao_especial'], inplace = True)

    #------------> Verificar:
    display(df)

,cnpj_basico,cnpj_ordem,cnpj_dv,cod_identificador_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,cod_motivo_situacao_cadastral,cod_cnae_fiscal_primaria,end_cep,end_uf,end_cod_municipio
7,22019650,0001,11,1,CUIABA CROMO,08,20180720,01,2539002,78140100,MT,9167
8,02269608,0004,94,2,NaN,08,20200909,01,4623109,78840000,MT,9779
13,22020365,0001,10,1,SANDRO LUIZ MUNARO,08,20151126,01,6821801,78450000,MT,9937
14,22020821,0001,22,1,POSTO LAVAGEM ALVES DE OLIVEIRA,08,20190529,01,4520005,78175000,MT,9129
15,22022798,0001,05,1,ACAI CLUB,08,20150806,01,5611203,78652000,MT,0131
...,...,...,...,...,...,...,...,...,...,...,...,...
910245,22009699,0001,93,1,AUTOCENTER MULTIMARCAS,08,20170925,01,4530703,78125044,MT,9167
910247,22011424,0001,94,1,PALATO SORVETES,08,20170926,01,1053800,78020973,MT,9067
910252,22014065,0001,29,1,STOP GRILL,08,20160115,01,5611203,78553004,MT,8985
910260,22016447,0001,91,1,CACA E PESCA MATRINXA,08,20160328,01,4763604,78578000,MT,1184


In [ ]:
#-------------------------------------------> Definindo tipos para as colunas


df = df.astype({
    'cnpj_basico':'int64',
    'cnpj_ordem':'int64',
    'cnpj_dv':'int64',
    'nome_fantasia':'object',
    'cod_identificador_matriz_filial':'category',
    'cod_situacao_cadastral':'category',
    'data_situacao_cadastral':'datetime64',
    'cod_motivo_situacao_cadastral':'category',
    'cod_cnae_fiscal_primaria':'category',
    'end_cep':'object',
    'end_uf':'category',
    'end_cod_municipio':'category'
    })

In [ ]:
#-------------> Ocorrencias a partir de MARÇO/2020
    
df = df.loc[(df['data_situacao_cadastral'] >= '2020-02-25') & (df['data_situacao_cadastral'] <= '2021-07-27')]

#--------------> Organizar por data

df.sort_values(by=['data_situacao_cadastral']) 


,cnpj_basico,cnpj_ordem,cnpj_dv,cod_identificador_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,cod_motivo_situacao_cadastral,cod_cnae_fiscal_primaria,end_cep,end_uf,end_cod_municipio
401993,33732741,1,16,1,NaN,08,2020-02-25,01,5320202,78550358,MT,8985
396063,32894519,1,57,1,GILBERTO FARIAS,08,2020-02-25,01,8599604,78550482,MT,8985
116643,27080984,1,14,1,FERNANDA MENDES,08,2020-02-26,01,9602502,78250000,MT,8999
401679,36297268,1,84,1,VOCE NA MODA,08,2020-02-26,01,4781400,78760000,MT,9083
203035,31028457,1,38,1,DECORDECK MOVEIS E DECORACOES,08,2020-02-26,01,4754701,78890000,MT,9907
...,...,...,...,...,...,...,...,...,...,...,...,...
317738,42072631,1,66,1,SEGUE LA,08,2021-07-27,01,5620104,78558418,MT,8985
895524,13783995,1,56,1,NaN,08,2021-07-27,01,4520002,78140050,MT,9167
851162,40009445,1,57,1,ANA LIGIA CAMPOS,08,2021-07-27,01,8592999,78050000,MT,9067
477850,37723357,1,8,1,RIBEIRO REPRESENTACOES,08,2021-07-27,01,7319002,78390000,MT,9033


## Validando colunas

In [ ]:
#Contagem de nulos

df.isna().sum()

cnpj_basico                            0
cnpj_ordem                             0
cnpj_dv                                0
cod_identificador_matriz_filial        0
nome_fantasia                      17081
cod_situacao_cadastral                 0
data_situacao_cadastral                0
cod_motivo_situacao_cadastral          0
cod_cnae_fiscal_primaria               0
end_cep                                0
end_uf                                 0
end_cod_municipio                      0
dtype: int64

In [ ]:
#STEP: ---------------------------------------------------> Validando colunas

print('Codigos de situação cadastral: ', df['cod_situacao_cadastral'].unique())
print('Codigos de municipios: ',df['end_cod_municipio'].unique())
print('Codigos da coluna estados: ', df['end_uf'].unique())
print('Codigos de motivos da situação cadastral: ', df['cod_motivo_situacao_cadastral'].unique())

Codigos de situação cadastral:  ['08']
Categories (1, object): ['08']
Codigos de municipios:  ['9779', '9183', '8985', '0127', '9151', ..., '0101', '1082', '1040', '9023', '9879']
Length: 141
Categories (141, object): ['9779', '9183', '8985', '0127', ..., '1082', '1040', '9023', '9879']
Codigos da coluna estados:  ['MT']
Categories (1, object): ['MT']
Codigos de motivos da situação cadastral:  ['01', '02', '31', '54', '66', '10', '05', '06', '15']
Categories (9, object): ['01', '02', '31', '54', ..., '10', '05', '06', '15']


## Tabelas auxiliares

In [ ]:
nome_motivos = ['cod_motivo_situacao_cadastral','Motivo']
df_motivos =  pd.read_csv("/content/drive/MyDrive/Tabelas apoio - CNPJ/Tabela de motivos .MOTICSV",sep=";",names=nome_motivos, dtype="object")

#display(df_motivos)

In [ ]:
nome_municipios = ['end_cod_municipio','Municipio']
df_municipios =  pd.read_csv("/content/drive/MyDrive/Tabelas apoio - CNPJ/Tabela de municipios .MUNICCSV",sep=";",names=nome_municipios, dtype="object")

#display(df_municipios)

In [ ]:
nome_cnae = ['cod_cnae_fiscal_primaria','Ramo da empresa']
df_cnae =  pd.read_csv("/content/drive/MyDrive/Tabelas apoio - CNPJ/tabela de CNAEs.CNAECSV",sep=";",names=nome_cnae,encoding="latin1", dtype="object")

#display(df_cnae)

## Colunas auxiliares


In [ ]:
#Nome dos municipios
df = pd.merge(df,df_municipios, how = 'left', on = 'end_cod_municipio')

#Nome dos motivos
df = pd.merge(df,df_motivos, how = 'left', on = 'cod_motivo_situacao_cadastral')

#Ramo da empresa
df = pd.merge(df,df_cnae, how = 'left', on = 'cod_cnae_fiscal_primaria')


#display(df)

In [ ]:
#Coluna Ano+Mes
df['Ano'] = df['data_situacao_cadastral'].dt.year.astype(str)
df['Mes'] = df['data_situacao_cadastral'].apply(lambda x: str(x)[5:7])

df['Ano/Mes'] = df[['Ano','Mes']].apply(''.join,axis=1).astype(int)

df.head(20)

,cnpj_basico,cnpj_ordem,cnpj_dv,cod_identificador_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,cod_motivo_situacao_cadastral,cod_cnae_fiscal_primaria,end_cep,end_uf,end_cod_municipio,Municipio,Motivo,Ramo da empresa,Ano/Mes,Ano,Mes
0,2269608,4,94,2,NaN,08,2020-09-09,01,4623109,78840000,MT,9779,CAMPO VERDE,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Comércio atacadista de alimentos para animais,202009,2020,09
1,22024175,1,71,1,NAZARI REPRESENTACOES,08,2021-06-23,01,4618499,78670000,MT,9183,SAO FELIX DO ARAGUAIA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Outros representantes comerciais e agentes do ...,202106,2021,06
2,22042298,1,35,1,SORVETERIA MUNDIAL,08,2020-02-27,01,4729699,78553228,MT,8985,SINOP,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Comércio varejista de produtos alimentícios em...,202002,2020,02
3,22058868,1,85,1,ELETRONICA CONCORD,08,2021-02-26,01,9521500,78550256,MT,8985,SINOP,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Reparação e manutenção de equipamentos eletroe...,202102,2021,02
4,4362488,3,10,2,PLANTAS & PROJETOS,08,2020-04-01,01,7490103,78665000,MT,0127,ALTO BOA VISTA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Serviços de agronomia e de consultoria às ativ...,202004,2020,04
5,22154346,1,87,1,RONDONOPOLIS PORTOES,08,2021-02-18,01,2542000,78715341,MT,9151,RONDONOPOLIS,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,"Fabricação de artigos de serralheria, exceto e...",202102,2021,02
6,18252508,2,23,2,MALTA SERVICOS,08,2020-09-15,01,8220200,78053734,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Atividades de teleatendimento,202009,2020,09
7,22182983,1,67,1,CENTER MOTOS RIO VERMELHO,08,2020-12-18,01,4541206,78735814,MT,9151,RONDONOPOLIS,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Comércio a varejo de peças e acessórios novos ...,202012,2020,12
8,22194897,1,74,1,AGUIA FLEX,08,2020-12-21,01,4754702,78048600,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Comércio varejista de artigos de colchoaria,202012,2020,12
9,22196481,1,95,1,SERVTEC,08,2020-07-05,01,4759899,78032035,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Comércio varejista de outros artigos de uso pe...,202007,2020,07


In [ ]:
print('Municipios no dataframe: ', df['Municipio'].unique())
print('CNAEs: ', df['cod_cnae_fiscal_primaria'].unique())
print('Motivos de encerramento ou baixa: ', df['Motivo'].unique())

Municipios no dataframe:  ['CAMPO VERDE' 'SAO FELIX DO ARAGUAIA' 'SINOP' 'ALTO BOA VISTA'
 'RONDONOPOLIS' 'CUIABA' 'PRIMAVERA DO LESTE' 'RIO BRANCO' 'ARAPUTANGA'
 'VARZEA GRANDE' 'ALTO GARCAS' 'ARENAPOLIS' 'ALTA FLORESTA' 'POCONE'
 'CANARANA' 'QUERENCIA' 'PONTES E LACERDA' 'NOVA UBIRATA'
 'CAMPOS DE JULIO' 'CACERES' 'SANTO AFONSO' 'TANGARA DA SERRA'
 'ROSARIO OESTE' 'PONTE BRANCA' 'GUARANTA DO NORTE' 'COLIDER'
 'LUCAS DO RIO VERDE' 'SAO JOSE DOS QUATRO MARCOS'
 'SANTO ANTONIO DO LEVERGER' 'NOVA CANAA DO NORTE' 'SORRISO' 'DIAMANTINO'
 'NOVA MONTE VERDE' 'BARRA DO GARCAS' 'PORTO DOS GAUCHOS' 'ITAUBA'
 'PARANAITA' 'CONFRESA' 'NOVA MUTUM' 'PEIXOTO DE AZEVEDO' 'ALTO ARAGUAIA'
 'NOBRES' 'AGUA BOA' 'NOVO SAO JOAQUIM' 'PEDRA PRETA' 'BRASNORTE'
 'MARCELANDIA' 'JUINA' 'NOVA XAVANTINA' 'PARANATINGA' 'JUSCIMEIRA'
 'VILA BELA DA SANTISSIMA TRINDADE' 'CANABRAVA DO NORTE'
 'SAO JOSE DO RIO CLARO' 'UNIAO DO SUL' 'SAPEZAL' 'TORIXOREU' 'ARIPUANA'
 'CAMPO NOVO DO PARECIS' 'APIACAS' "MIRASSOL D'OESTE" 'AL

# Informações & insights

In [ ]:
qtd_baixa = df['cnpj_basico'].nunique()
print('Quantidade de empresas baixadas: ', qtd_baixa)

qtd_municipios = df['end_cod_municipio'].nunique()
print('Quantidade de municipios no dataframe: ', qtd_municipios )

Qtd_ramos_afetados = df['Ramo da empresa'].nunique()
print('Quantidade de ramos economicos afetados: ', Qtd_ramos_afetados )

Quantidade de empresas baixadas:  44059
Quantidade de municipios no dataframe:  141
Quantidade de ramos economicos afetados:  819


In [ ]:
# Contagem de baixas de empresas por município
topcinco_baixas = df['Municipio'].value_counts().nlargest(5)
quantidade_ocorrencias_topcinco = df['Municipio'].value_counts().nlargest(5).sum()

print(topcinco_baixas)
print('Quantidade total de baixas no TOP 5 municipios: ' + str(quantidade_ocorrencias_topcinco))



CUIABA           8466
VARZEA GRANDE    2888
RONDONOPOLIS     2632
SINOP            2041
SORRISO          1121
Name: Municipio, dtype: int64
Quantidade total de baixas no TOP 5 municipios: 17148


In [ ]:
# Representatividade do top 10 sobre o todo

percentual_rep_topcinco = float(int(quantidade_ocorrencias_topcinco) / int(df['cnpj_basico'].nunique()))
percentual_rep_topcinco = '{0:.0%}'.format(percentual_rep_topcinco)
print('O top 5 representa ' + str(percentual_rep_topcinco) + ' das ocorrências no estado do ' + str(df['end_uf'].unique()))

O top 5 representa 39% das ocorrências no estado do ['MT']
Categories (1, object): ['MT']


In [ ]:
#Distribuição dos motivos

qtd_por_motivos =  df['Motivo'].value_counts()
qtd_por_motivos

EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA                                           44713
INCORPORACAO                                                                                 83
INAPTIDAO                                                                                    56
INEXISTENCIA DE FATO                                                                         11
ENCERRAMENTO DA FALENCIA                                                                     10
EXTINCAO PELO ENCERRAMENTO DA LIQUIDACAO JUDICIAL                                             3
EXTINCAO-UNIFICACAO DA FILIAL                                                                 3
EXTINCAO - TRATAMENTO DIFERENCIADO DADO AS ME E EPP (LEI COMPLEMENTAR NUMERO 123/2006)        1
ENCERRAMENTO DA LIQUIDACAO                                                                    1
Name: Motivo, dtype: int64

In [ ]:
#Distribuição dos ramos mais afetados no estado de xxx

Top_cinco_ramos = df['Ramo da empresa'].value_counts().nlargest(5)
Top_cinco_ramos

Atividades de organizações políticas                                                                                            12903
Comércio varejista de artigos do vestuário e acessórios                                                                          2423
Promoção de vendas                                                                                                               1099
Lanchonetes, casas de chá, de sucos e similares                                                                                   996
Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns      805
Name: Ramo da empresa, dtype: int64

In [ ]:
# Investigar ramo Atividades de organizações políticas

tabela_invest = df.loc[(df['Ramo da empresa'] == 'Atividades de organizações políticas')]


tabela_invest_mes = tabela_invest.groupby('Ano/Mes')['cnpj_basico'].count()
tabela_invest_mes
#tabela_invest.head()

Ano/Mes
202003        1
202005        1
202006        2
202009       32
202102    12830
202103       32
202104        3
202106        1
202107        1
Name: cnpj_basico, dtype: int64

In [ ]:
# Datas com maior baixa de empresas

Top_cinco_datas = df['data_situacao_cadastral'].value_counts().nlargest(5)
Top_cinco_datas

2021-02-28    12607
2021-07-19      330
2021-08-31      239
2021-08-25      194
2021-05-31      193
Name: data_situacao_cadastral, dtype: int64

In [ ]:
# Análise de baixa por datas

#quantidade média de baixas por mês
qtd_meses_periodo = df['Ano/Mes'].nunique()
qtd_media_baixas_mes = qtd_baixa/qtd_meses_periodo
print('A quantidade média de baixas por mês foi: ' + str(qtd_media_baixas_mes))


#Histograma
tabela_baixas_por_mes_ano = df.groupby('Ano/Mes')['cnpj_basico'].count()

display(tabela_baixas_por_mes_ano)

A quantidade média de baixas por mês foi: 2447.722222222222


Ano/Mes
202002      347
202003     2020
202004     1447
202005     1807
202006     1666
202007     1840
202008     1699
202009     1554
202010     1674
202011     1514
202012     1734
202101     2163
202102    15027
202103     2411
202104     1822
202105     2290
202106     2079
202107     1787
Name: cnpj_basico, dtype: int64

In [ ]:
# inteligencia de mercado
tab_inteligencia = df.groupby(['Municipio','end_bairro','Ramo da empresa'])['cnpj_basico'].count().reset_index()
tab_inteligencia = tab_inteligencia.loc[(tab_inteligencia['Ramo da empresa'] != 'Atividades de organizações políticas')]
tab_inteligencia['Ramo da empresa'] = tab_inteligencia['Ramo da empresa'].astype('category')
tab_inteligencia = tab_inteligencia.sort_values('cnpj_basico', ascending=False)
# tab_inteligencia

# inteligencia de mercado - tabela com maior baixa de empresas por ramo, bairro e municipio
tab_top_ramo = tab_inteligencia.drop_duplicates('Ramo da empresa', keep = 'first')
tab_top_ramo.head(30)

In [ ]:
cidades = ['CUIABA', 'VARZEA GRANDE', 'RONDONOPOLIS', 'SINOP', 'CACERES']
for cidade in cidades:
    display(cidade)
    display(df.loc[(df['Municipio']==cidade)&(df['Ramo da empresa']!='Atividades de organizações políticas')].groupby('Ramo da empresa').count().sort_values('cnpj_ordem', ascending=False)['cnpj_basico'][0:6])

'CUIABA'

Ramo da empresa
Comércio varejista de artigos do vestuário e acessórios                            571
Promoção de vendas                                                                 335
Restaurantes e similares                                                           221
Cabeleireiros, manicure e pedicure                                                 216
Lanchonetes, casas de chá, de sucos e similares                                    210
Fornecimento de alimentos preparados preponderantemente para consumo domiciliar    198
Name: cnpj_basico, dtype: int64

'VARZEA GRANDE'

Ramo da empresa
Comércio varejista de artigos do vestuário e acessórios                                                                           167
Promoção de vendas                                                                                                                 90
Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns       84
Comércio varejista de produtos alimentícios em geral ou especializado em produtos alimentícios não especificados anteriormente     74
Restaurantes e similares                                                                                                           61
Lanchonetes, casas de chá, de sucos e similares                                                                                    55
Name: cnpj_basico, dtype: int64

'RONDONOPOLIS'

Ramo da empresa
Comércio varejista de artigos do vestuário e acessórios                                                                151
Promoção de vendas                                                                                                      96
Transporte rodoviário de carga, exceto produtos perigosos e mudanças, intermunicipal, interestadual e internacional     69
Obras de alvenaria                                                                                                      69
Lanchonetes, casas de chá, de sucos e similares                                                                         57
Atividades de fornecimento de infra-estrutura de apoio e assistência a paciente no domicílio                            57
Name: cnpj_basico, dtype: int64

'SINOP'

Ramo da empresa
Comércio varejista de artigos do vestuário e acessórios                                                                132
Promoção de vendas                                                                                                      97
Lanchonetes, casas de chá, de sucos e similares                                                                         64
Transporte rodoviário de carga, exceto produtos perigosos e mudanças, intermunicipal, interestadual e internacional     57
Obras de alvenaria                                                                                                      34
Restaurantes e similares                                                                                                33
Name: cnpj_basico, dtype: int64

'CACERES'

Ramo da empresa
Comércio varejista de artigos do vestuário e acessórios                                                                         54
Lanchonetes, casas de chá, de sucos e similares                                                                                 31
Promoção de vendas                                                                                                              30
Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns    29
Cabeleireiros, manicure e pedicure                                                                                              18
Fornecimento de alimentos preparados preponderantemente para consumo domiciliar                                                 17
Name: cnpj_basico, dtype: int64

In [ ]:
cidades = ['CUIABA', 'VARZEA GRANDE', 'RONDONOPOLIS', 'SINOP', 'CACERES']
for cidade in cidades:
    display(cidade)
    display(df.loc[(df['Municipio']==cidade)&(df['Ramo da empresa']!='Atividades de organizações políticas')].groupby('Ramo da empresa').count().sort_values('cnpj_ordem', ascending=False)[0:6])

'CUIABA'

,cnpj_basico,cnpj_ordem,cnpj_dv,cod_identificador_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,cod_motivo_situacao_cadastral,cod_cnae_fiscal_primaria,end_bairro,end_cep,end_uf,end_cod_municipio,Municipio,Motivo,Ano,Mes,Ano/Mes
Ramo da empresa,,,,,,,,,,,,,,,,,,
Comércio varejista de artigos do vestuário e acessórios,571,571,571,571,520,571,571,571,571,571,571,571,571,571,571,571,571,571
Promoção de vendas,335,335,335,335,272,335,335,335,335,335,335,335,335,335,335,335,335,335
Restaurantes e similares,221,221,221,221,208,221,221,221,221,221,221,221,221,221,221,221,221,221
"Cabeleireiros, manicure e pedicure",216,216,216,216,176,216,216,216,216,216,216,216,216,216,216,216,216,216
"Lanchonetes, casas de chá, de sucos e similares",210,210,210,210,196,210,210,210,210,210,210,210,210,210,210,210,210,210
Fornecimento de alimentos preparados preponderantemente para consumo domiciliar,198,198,198,198,181,198,198,198,198,198,198,198,198,198,198,198,198,198


'VARZEA GRANDE'

,cnpj_basico,cnpj_ordem,cnpj_dv,cod_identificador_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,cod_motivo_situacao_cadastral,cod_cnae_fiscal_primaria,end_bairro,end_cep,end_uf,end_cod_municipio,Municipio,Motivo,Ano,Mes,Ano/Mes
Ramo da empresa,,,,,,,,,,,,,,,,,,
Comércio varejista de artigos do vestuário e acessórios,167,167,167,167,149,167,167,167,167,167,167,167,167,167,167,167,167,167
Promoção de vendas,90,90,90,90,76,90,90,90,90,90,90,90,90,90,90,90,90,90
"Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns",84,84,84,84,76,84,84,84,84,84,84,84,84,84,84,84,84,84
Comércio varejista de produtos alimentícios em geral ou especializado em produtos alimentícios não especificados anteriormente,74,74,74,74,64,74,74,74,74,74,74,74,74,74,74,74,74,74
Restaurantes e similares,61,61,61,61,55,61,61,61,61,61,61,61,61,61,61,61,61,61
"Lanchonetes, casas de chá, de sucos e similares",55,55,55,55,50,55,55,55,55,55,55,55,55,55,55,55,55,55


'RONDONOPOLIS'

,cnpj_basico,cnpj_ordem,cnpj_dv,cod_identificador_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,cod_motivo_situacao_cadastral,cod_cnae_fiscal_primaria,end_bairro,end_cep,end_uf,end_cod_municipio,Municipio,Motivo,Ano,Mes,Ano/Mes
Ramo da empresa,,,,,,,,,,,,,,,,,,
Comércio varejista de artigos do vestuário e acessórios,151,151,151,151,131,151,151,151,151,151,151,151,151,151,151,151,151,151
Promoção de vendas,96,96,96,96,74,96,96,96,96,96,96,96,96,96,96,96,96,96
"Transporte rodoviário de carga, exceto produtos perigosos e mudanças, intermunicipal, interestadual e internacional",69,69,69,69,62,69,69,69,69,69,69,69,69,69,69,69,69,69
Obras de alvenaria,69,69,69,69,42,69,69,69,69,69,69,69,69,69,69,69,69,69
"Lanchonetes, casas de chá, de sucos e similares",57,57,57,57,53,57,57,57,57,57,57,57,57,57,57,57,57,57
Atividades de fornecimento de infra-estrutura de apoio e assistência a paciente no domicílio,57,57,57,57,23,57,57,57,57,57,57,57,57,57,57,57,57,57


'SINOP'

,cnpj_basico,cnpj_ordem,cnpj_dv,cod_identificador_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,cod_motivo_situacao_cadastral,cod_cnae_fiscal_primaria,end_bairro,end_cep,end_uf,end_cod_municipio,Municipio,Motivo,Ano,Mes,Ano/Mes
Ramo da empresa,,,,,,,,,,,,,,,,,,
Comércio varejista de artigos do vestuário e acessórios,132,132,132,132,119,132,132,132,132,132,132,132,132,132,132,132,132,132
Promoção de vendas,97,97,97,97,77,97,97,97,97,97,97,97,97,97,97,97,97,97
"Lanchonetes, casas de chá, de sucos e similares",64,64,64,64,61,64,64,64,64,64,64,64,64,64,64,64,64,64
"Transporte rodoviário de carga, exceto produtos perigosos e mudanças, intermunicipal, interestadual e internacional",57,57,57,57,46,57,57,57,57,57,57,57,57,57,57,57,57,57
Obras de alvenaria,34,34,34,34,29,34,34,34,34,34,34,34,34,34,34,34,34,34
Restaurantes e similares,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33


'CACERES'

,cnpj_basico,cnpj_ordem,cnpj_dv,cod_identificador_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,cod_motivo_situacao_cadastral,cod_cnae_fiscal_primaria,end_bairro,end_cep,end_uf,end_cod_municipio,Municipio,Motivo,Ano,Mes,Ano/Mes
Ramo da empresa,,,,,,,,,,,,,,,,,,
Comércio varejista de artigos do vestuário e acessórios,54,54,54,54,51,54,54,54,54,54,54,54,54,54,54,54,54,54
"Lanchonetes, casas de chá, de sucos e similares",31,31,31,31,25,31,31,31,31,31,31,31,31,31,31,31,31,31
Promoção de vendas,30,30,30,30,23,30,30,30,30,30,30,30,30,30,30,30,30,30
"Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns",29,29,29,29,28,29,29,29,29,29,29,29,29,29,29,29,29,29
"Cabeleireiros, manicure e pedicure",18,18,18,18,17,18,18,18,18,18,18,18,18,18,18,18,18,18
Fornecimento de alimentos preparados preponderantemente para consumo domiciliar,17,17,17,17,13,17,17,17,17,17,17,17,17,17,17,17,17,17


In [ ]:
for cidade in cidades:
    display (df[df['Municipio']==cidade]).value_counts().

,cnpj_basico,cnpj_ordem,cnpj_dv,cod_identificador_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,cod_motivo_situacao_cadastral,cod_cnae_fiscal_primaria,end_bairro,end_cep,end_uf,end_cod_municipio,Municipio,Motivo,Ramo da empresa,Ano,Mes,Ano/Mes
6,18252508,2,23,2,MALTA SERVICOS,08,2020-09-15,01,8220200,MORADA DO OURO II,78053734,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Atividades de teleatendimento,2020,09,202009
8,22194897,1,74,1,AGUIA FLEX,08,2020-12-21,01,4754702,JARDIM ALVORADA,78048600,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Comércio varejista de artigos de colchoaria,2020,12,202012
9,22196481,1,95,1,SERVTEC,08,2020-07-05,01,4759899,GOIABEIRAS,78032035,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Comércio varejista de outros artigos de uso pe...,2020,07,202007
10,22199597,1,88,1,SORVETERIA LEVE DELICIA,08,2020-10-22,01,5611203,RESIDENCIAL BRASIL 21,78070971,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,"Lanchonetes, casas de chá, de sucos e similares",2020,10,202010
13,22236159,1,42,1,JF JARDINAGEM,08,2021-03-18,01,8130300,SOL NASCENTE,78051623,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Atividades paisagísticas,2021,03,202103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44857,21410163,1,12,1,PRIME,08,2020-11-25,01,6619302,CENTRO NORTE,78005500,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Correspondentes de instituições financeiras,2020,11,202011
44859,21458890,1,50,1,ESTETICA SANTE,08,2020-10-13,01,9602502,DUQUE DE CAXIAS,78043272,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Atividades de estética e outros serviços de cu...,2020,10,202010
44862,21490460,1,15,1,KAR ACESSORIOS,08,2020-04-29,01,4530701,DOM AQUINO,78015020,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Comércio por atacado de peças e acessórios nov...,2020,04,202004
44878,21940516,1,96,1,MERKATUS REPRESENTACOES,08,2020-05-21,01,7319003,BOSQUE DA SAUDE II,78050200,MT,9067,CUIABA,EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA,Marketing direto,2020,05,202005


AttributeError: ignored